In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
import random
random.seed(42)
import warnings
warnings.filterwarnings("ignore")

In [41]:
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
def makeDataset(fold):
    base = pd.read_csv("/content/drive/MyDrive/CFAssignmentsDataset/u" + str(fold) + ".base", sep = "\t", names = columns, encoding = "latin-1")
    test = pd.read_csv("/content/drive/MyDrive/CFAssignmentsDataset/u" + str(fold) + ".test", sep = "\t", names = columns, encoding = "latin-1")
    
    return base, test

def makeYandRMatrix(data):
    totalUsers = len(data['user_id'].unique())
    totalMovies = 1682
    Y = np.zeros([totalUsers, totalMovies])
    R = np.zeros([totalUsers, totalMovies])

    for i in range(data.shape[0]):
        userId = data['user_id'][i]-1
        movieId = data['movie_id'][i]-1
        curRating = data['rating'][i]

        Y[userId][movieId] = curRating 
        R[userId][movieId] = 1

    return Y, R

def train(Y, R, t, max_iters = 10):
    nmaes = []
    X = np.random.uniform(size=(943, 1682), low=1, high=5)

    for i in range(max_iters):
        B = X + Y - np.multiply(R, X)
        U, S, V = np.linalg.svd(B, full_matrices=False)
        S = np.maximum(0, S - 0.05)
        X = np.dot(np.dot(U, np.diag(S)), V)
        rPred = predictRating(X, t)
        nmaes.append(nmae(t, rPred))
           
    return X, nmaes

def predictRating(X, test):
    ratingsPred = []

    for i in range(test.shape[0]):
        userId = test['user_id'][i]-1
        movieId = test['movie_id'][i]-1

        ratingPredicted = X[userId][movieId]
        ratingsPred.append(round(ratingPredicted))

    return ratingsPred

def nmae(test, ratingsPred):
    inds = np.nonzero(ratingsPred)
    ratingsTrue = np.array(test['rating'])[inds]
    ratingsPred = np.array(ratingsPred)[inds]
    error = mae(ratingsTrue, ratingsPred)/4
    return error


In [36]:
f = 1
b, t = makeDataset(f)
Y, R = makeYandRMatrix(b)

In [42]:
X, nmaes = train(Y, R, t, 50)
print(min(nmaes))

0.3415125
